In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time

class InstagramBot:
    def __init__(self, username, password):
        # Set up the WebDriver
        self.driver = webdriver.Chrome()
        self.username = username
        self.password = password
    
    def login(self):
        # Navigate to the Instagram login page
        self.driver.get('https://www.instagram.com/')

        # Locate the username/email field and enter the input
        WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.NAME, 'username'))
        ).send_keys(self.username)

        # Locate the password field and enter the input
        self.driver.find_element(By.NAME, 'password').send_keys(self.password)

        # Submit the form by clicking the "Log in" button
        self.driver.find_element(By.XPATH, '//button[@type="submit"]').click()

        # Wait for an element that appears only after successful login
        try:
            WebDriverWait(self.driver, 60).until(
                EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "some-class-after-login")]'))
            )
            print("Successfully logged in!")
        except Exception as e:
            print(f"Login failed or the element was not found: {e}")
    
    def navigate_to_profile(self, profile_url):
        # Redirect to the desired profile
        self.driver.get(profile_url)
        print(f"Navigated to {profile_url}")

        # Wait for up to 30 seconds to allow the page to load completely
        WebDriverWait(self.driver, 30).until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(@href, '/followers/')]"))
        )
        print("Page fully loaded!")

    def fetch_followers(self):
        # Locate the Followers link and click it
        try:
            followers_link = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, '/followers/')]"))
            )
            followers_link.click()
            print("Clicked the followers link!")
        except Exception as e:
            print(f"Failed to click the followers link: {e}")
            return
        
        # Wait for the followers modal to load
        time.sleep(5)  # Adjust sleep time as needed

        # Get the page source and parse it with BeautifulSoup
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')

        # Find all followers based on the provided class
        followers_divs = soup.find_all('div', class_='x9f619 xjbqb8w x78zum5 x168nmei x13lgxp2 x5pf9jr xo71vjh x1uhb9sk x1plvlek xryxfnj x1c4vz4f x2lah0s xdt5ytf xqjyukv x1qjc9v5 x1oa3qoh x1nhvcw1')

        followers_list = []
        for follower_div in followers_divs:
            profile_link_tag = follower_div.find('a')
            profile_link = profile_link_tag['href'] if profile_link_tag else None
            profile_image = profile_link_tag.find('img')['src'] if profile_link_tag and profile_link_tag.find('img') else None
            
            name_tag = follower_div.find('div', class_='x9f619 xjbqb8w x78zum5 x168nmei x13lgxp2 x5pf9jr xo71vjh x1uhb9sk x1plvlek xryxfnj x1c4vz4f x2lah0s xdt5ytf xqjyukv x1qjc9v5 x1oa3qoh x1nhvcw1')
            name = name_tag.text if name_tag else None

            follower_data = {
                'Profile Link': profile_link,
                'Profile Image URL': profile_image,
                'Name': name
            }
            
            # Print follower data to the console
            print(f"Profile Link: {follower_data['Profile Link']}")
            print(f"Profile Image URL: {follower_data['Profile Image URL']}")
            print(f"Name: {follower_data['Name']}")
            print('---')
            
            # Add to the list to save later
            followers_list.append(follower_data)
        
        # Save followers data to a file
        with open('followers_list2.txt', 'w') as f:
            for follower in followers_list:
                f.write(f"Profile Link: {follower['Profile Link']}\n")
                f.write(f"Profile Image URL: {follower['Profile Image URL']}\n")
                f.write(f"Name: {follower['Name']}\n")
                f.write('\n')
        
        print(f'Fetched and saved {len(followers_list)} followers.')

    def close_browser(self):
        self.driver.quit()

# Prompt user for input
username = input("Enter your Instagram username, phone number, or email: ")
password = input("Enter your Instagram password: ")

# Initialize the bot and perform login
bot = InstagramBot(username, password)
bot.login()

# Specify the profile URL
profile_url = 'https://www.instagram.com/virtuosoltd/'
bot.navigate_to_profile(profile_url)

# Fetch and print followers
bot.fetch_followers()

# Close the browser
bot.close_browser()